In [2]:
import numpy as np
from numpy.polynomial import polynomial as P
import math
import matplotlib.pyplot as plt

In [2]:
n=10
bound=[0,1]

In [4]:
class poly:
    def __init__(self,pts=None,vls=None,arr=None):
        if(arr is not None):
            self.deg=len(arr)-1
            self.arr=self.arr
        if(pts is not None):
            self.get_poly(pts,vls)
    def eval(self,x):
        val=np.ones(x.shape)
        ans=np.zeros(x.shape)
        for i in range(self.deg+1):
            ans+=val*self.arr[i]
            val*=x
        return ans
    def der(self,x):
        val=np.ones(x.shape)
        ans=np.zeros(x.shape)
        for i in range(1,self.deg+1):
            ans+=val*self.arr[i]*i
            val*=x
        return ans
    def roots(self):
        return P.polyroots(tuple(self.arr))
    def optims(self):
        #print((self.arr*np.arange(len(self.arr)))[1:])
        return P.polyroots(tuple((self.arr*np.arange(len(self.arr)))[1:]))
    def get_poly(self,pts,vls):
        self.deg=len(pts)-1
        mtx=[]
        for i in range(self.deg+1):
          mtx.append(np.power(pts,i))
        mtx=np.array(mtx)
        self.arr=np.squeeze(np.matmul(np.linalg.inv(mtx.T),np.reshape(vls,(len(vls),1))))
    def solve(self,x):
        self.arr[-1]-=x
        toret=P.polyroots(tuple(self.arr))
        self.arr[-1]+=x
        return toret
print(poly(pts=[1,2,3],vls=[0,1,2]).eval(np.array([1,2])))


[0. 1.]


In [13]:
A=1.128
def fun(x):
    return np.sin(x)+x**2-1

In [5]:
#cheebyshev

pts=bound[0]+(bound[1]-bound[0])*np.cos((2*np.arange(n+1)+1)/(2*n+1))
vals=fun(pts)
to_eval=0.15
print(poly(pts,vals).eval(to_eval))

0.14951924365852354


In [7]:
#piecewilse linear
pts=[-1,0,1]
vals=fun(pts)
to_eval_lst=[-0.75,0.75]
deg=2
new_pts=[pts[i*(deg):(i+1)*(deg)+1] for i in range(len(pts)//deg)]
arr=[]
for to_eval in to_eval_lst:
    for t in range(len(new_pts)):
        i=new_pts[t]
        #print(i)
        if(to_eval>=i[0] and to_eval<=i[-1]):
            arr.append(poly(i,vals[t*(deg):(t+1)*(deg)+1]).eval(np.array([to_eval])))
print(arr)
        
       

[array([0.64443219]), array([0.64443219])]


In [9]:
#iterative methods::

def bisection(f,lb,hb,eps=1e-5,round=0):
    #precon:
    #|r-x|<eps::
    val_l=f(lb)
    val_h=f(hb)
    val=f((lb+hb)/2)
    bw_wal=(lb+hb)/2
    print(f"x{round+1}: {f(bw_wal)}")
    if(val_h*val_l>0):
        print(f"Not possibel to search {lb}  {hb}")
        return 0
    if(hb-lb<=2*eps):
        return bw_wal
    if(val*val_l<0):
        return bisection(f,lb,(lb+hb)/2,eps,round+1)
    else:
        return bisection(f,(lb+hb)/2,hb,eps,round+1)
def iter(lb,hb,eps):
    #|r-x|<eps::
    return math.ceil(np.log((hb-lb)/eps)/np.log(2))
print(iter(0,1,1e-4))
bisection(fun,0,1,1e-4)


14
x1: 0.7788007830714049
Not possibel to search 0  1


0

In [11]:
#Regula-Falsi method:
bg_arr=[]
def RF(f,lb,hb,eps=1e-5,N=100,round=0):
    #precon:
    #|r-x|<eps::
    global bg_arr
    val_l=f(lb)
    val_h=f(hb)
    bw_wal=(lb*(val_h)-hb*(val_l))/(val_h-val_l)
    val=f(bw_wal)
    bg_arr.append(bw_wal)
    print(f"x{round+1}: {bw_wal}")
    if(val_h*val_l>0):
        print(f"Not possibel to search {lb}  {hb}")
        return 0
    if(round>=N):
        if(abs(bg_arr[round]-bg_arr[round-N])<=eps):
           return bw_wal
    if(val*val_l<0):
        return RF(f,lb,bw_wal,eps,N,round+1)
    else:
        return RF(f,bw_wal,hb,eps,N,round+1)
RF(fun,0,4.5,1e-10,5)

x1: 3.3333333333333335
x2: 1.8750000000000004
x3: 1.2121212121212122
x4: 1.0443037974683544
x5: 1.0089399744572158
x6: 1.0017911975435008
x7: 1.000358367890237
x8: 1.0000716787155173
x9: 1.00001433594862
x10: 1.000002867197945
x11: 1.0000005734399176
x12: 1.0000001146879969
x13: 1.0000000229376
x14: 1.0000000045875201
x15: 1.0000000009175039
x16: 1.0000000001835008
x17: 1.0000000000367002
x18: 1.0000000000073404
x19: 1.000000000001468
x20: 1.0000000000002938
x21: 1.000000000000059
x22: 1.0000000000000118


1.0000000000000118

In [14]:
#secant method::
bg_arr=[]
def der(f,x,eps=1e-30):
    return (f(x)-f(x-eps))/eps
def secant(f,lb,hb,eps=1e-5,N=100,round=0):
    #precon:
    #|r-x|<eps::
    if(lb>hb):
        t=lb
        lb=hb
        hb=t
    global bg_arr
    val_l=f(lb)
    val_h=f(hb)
    if(val_l==val_h):
        print("parallel line")
        return 0
    bw_wal=(lb*(val_h)-hb*(val_l))/(val_h-val_l)
    val=f(bw_wal)
    bg_arr.append(bw_wal)
    print(f"x{round+1}: {bw_wal}")
    if(round>=N):
        if(abs(bg_arr[round]-bg_arr[round-N])<=eps):
           return bw_wal
    return secant(f,bw_wal,hb,eps,N,round+1)
secant(fun,0,1,1e-10,5)

x1: 0.5430441251857795
x2: 0.6266225470848197
x3: 0.6356849973811681
x4: 0.636624553706821
x5: 0.6367215022781226
x6: 0.6367315010615463
x7: 0.6367325322331701
x8: 0.636732638577045
x9: 0.636732649544195
x10: 0.6367326506752273
x11: 0.6367326507918697
x12: 0.6367326508038988
x13: 0.6367326508051393
x14: 0.6367326508052673
x15: 0.6367326508052804
x16: 0.636732650805282


0.636732650805282

In [4]:
#newton raphson::
def der(f,x,order=1,eps=1e-5):
    if(order==1):
      return (f(x)-f(x-eps))/eps
    else:
      return (der(f,x,order-1,eps=eps)-der(f,x-order*eps,order-1,eps=eps))/(order*eps)
#print(der(fun,3,1,1e-8))
bg_arr=[]
def NF(f,x,der_f=der,eps=1e-10,N=5):
   x_comp=x
   bg_arr.append(x)
   iter=0
   while(iter<N or abs(bg_arr[iter]-bg_arr[iter-N])>eps):
      iter+=1
      x_comp=x_comp-f(x_comp)/der_f(f,x_comp,eps=1e-8)
      bg_arr.append(x_comp)
      print(f"x{iter}:{x_comp} val{iter}:{f(x_comp)}")

#NF(fun,A)
def ans(x):
   return (x**2-64)/(2*x-34)
#ans(A)
   

In [16]:
P=0.125
def g(x):
    return 1/(x+P)
    pass#x=g(x) is alternate for f(x)::
def plot(fun,start, iter):
    x_1=np.arange(start-1,start+1,0.1)
    y_1=fun(x_1)
    plt.plot(x_1,y_1,'b-',x_1,x_1,'r-')
    x_val=[start]
    for i in range(iter):
       x_val.append(fun(x_val[-1]))
    plt.plot(x_val,fun(x_val),'y^')
    plt.show()
def iters(fun,x0,lam,eps=1e-5):
    x1=fun(x0)
    gt=eps/abs(x1-x0)
    mi=np.log(gt*(1-lam))/np.log(lam)
    return math.ceil(mi)

def fp(f,x,g=None,N=5,eps=1e-10):
    def g_p(x):
        return np.sqrt(x)
    if(g is None):
        g=g_p
    bg_arr=[x]
    iter=0
    while(iter<N or abs(bg_arr[iter]-bg_arr[iter-N])>eps):
        x=g(x)
        bg_arr.append(x)
        iter+=1
        #print(f"x{iter}: {x} , val{iter}: {f(x)}")
    return x

#iters(fun,0.5,0.5)
#plot(np.sqrt,0.1,10)
#fp(fun,10,g)


        

In [18]:
#   Modified newtons method:;
def f(x):
   return np.power(x-1,3)
def f1(f,x,eps=1e-5):
   return 3*np.power(x-1,2)
def f2(f,x,eps=1e-5):
   return 6*((x-1))
def MNF(f,x,der=None,dder=None,eps=1e-5,N=5):
   bg_arr=[]
   x_comp=x
   bg_arr.append(x)
   iter=0
   while(iter<N or abs(bg_arr[iter]-bg_arr[iter-N])>eps):
      iter+=1
      x_comp=x_comp-der(f,x_comp)*f(x_comp)/(der(f,x_comp)**2-dder(f,x_comp)*f(x_comp))
      bg_arr.append(x_comp)
      print(f"x{iter}:{x_comp} val{iter}:{f(x_comp)}")
MNF(f,200,f1,f2)
NF(f,200,f1)

x1:1.0 val1:0.0
x2:nan val2:nan
x3:nan val3:nan
x4:nan val4:nan
x5:nan val5:nan
x1:133.66666666666669 val1:2334992.296296297
x2:89.44444444444446 val2:691849.569272977
x3:59.962962962962976 val3:204992.464969771
x4:40.308641975308646 val4:60738.508139191385
x5:27.205761316872433 val5:17996.59500420486


/var/folders/t9/dvv1y5pj7w3_yd12t_wd2fq80000gn/T/ipykernel_6549/2786825949.py:15: RuntimeWarning: invalid value encountered in double_scalars
  x_comp=x_comp-der(f,x_comp)*f(x_comp)/(der(f,x_comp)**2-dder(f,x_comp)*f(x_comp))


In [19]:
#modified newtons given multiplicity::
bg_arr=[]
def MNF1(f,x,m=1,der_f=der,eps=1e-5,N=5):
   x_comp=x
   bg_arr.append(x)
   iter=0
   while(iter<N or abs(bg_arr[iter]-bg_arr[iter-N])>eps):
      iter+=1
      x_comp=x_comp-m*f(x_comp)/der_f(f,x_comp,eps=1e-8)
      bg_arr.append(x_comp)
      print(f"x{iter}:{x_comp} val{iter}:{f(x_comp)}")
MNF1(f,0.75,2,f1)

x1:0.9166666666666666 val1:-0.0005787037037037044
x2:0.9722222222222222 val2:-2.143347050754461e-05
x3:0.9907407407407407 val3:-7.938322410201802e-07
x4:0.9969135802469136 val4:-2.9401194111858528e-08
x5:0.9989711934156379 val5:-1.088933115253902e-09
x6:0.9996570644718793 val6:-4.033085612051489e-11
x7:0.9998856881572931 val7:-1.4937354118723727e-12
x8:0.999961896052431 val8:-5.532353377305084e-14
x9:0.9999872986841437 val9:-2.049019769354343e-15
x10:0.9999957662280479 val10:-7.588962108918794e-17
x11:0.9999985887426827 val11:-2.8107267067858433e-18
x12:0.9999995295808942 val12:-1.0410098914021642e-19
x13:0.9999998431936314 val13:-3.855592187648543e-21
x14:0.9999999477312105 val14:-1.4279971065364975e-22
x15:0.9999999825770701 val15:-5.28887820605916e-24


In [8]:
#Integration(numerical)::
A=2.4
B=4.9
def transform_simple(pts,f,deg=None,):
      return f(pts[:,0])
def transform_midpt(pts,f,deg=None):
    return f((pts[:,0]+pts[:,1])/2)
def transform_trapezoid(pts,f,deg=None):
    return [((f(pt[0])+f(pt[1]))/2) for pt in pts]
def transform_simpsons(pts,f,deg=None):
    return [(f(pt[0])+4*(f((pt[0]+pt[1])/2))+f(pt[1]))/6 for pt in pts]
def error(f,a,b,N=10,deg=2,pts=None,meth=transform_simpsons):
    if(pts is None):
        pts=np.array([[a+((b-a)/N)*i,a+((b-a)/N)*(i+1)] for i in range(N)])
        pts_t=meth(pts,f,deg)
    ans=0
    for i in range(len(pts)):
        ans+=pts_t[i]*(pts[i][1]-pts[i][0])
    print(f"integeral:{ans}")
    return ans
def uc(f,pts,w):
    #undetermined coeffieciens::
    ans=0
    for i in range(pts):
        ans+=pts[i]*w[i]
    print(f"Method of undermined coeffiecients:{ans}")
def f(x):
    return A*0.25*np.power(x,4)+B*0.5*(np.power(x,2))
#error(f,A,B,1)
    

In [9]:
#guassian rule::
def transform_quad_guass(pts,f,deg=2):
    arr=[]
    for pt in pts:
        pts_p,w_p=weights(pt[0],pt[1],deg)
        ans=0
        for i in range(len(pts_p)):
            ans+=w_p[i]*f(pts_p[i])
        arr.append(ans)
    return arr

def weights(a,b,deg):
    pts,wts=np.polynomial.legendre.leggauss(deg)
    #print((b-a)*pts+a,wts/2)
    return ((b-a)*pts+a+b)/2,wts/2
#weights(0,10,2)
#error(lambda x:np.exp(-x**2),1,3,1)


In [73]:
#derivatives:
def der_new(f,a,h=1e-6,type="0"):
    if type=='+':
      #forward
      return (f(a+h)-f(a))/h
    elif(type=='-'):
      #backward
      return (f(a)-f(a-h))/h
    else:
      #central difference::
      return (f(a+h)-f(a-h))/(2*h)
der_new(lambda x: np.exp(x**2),1.7,h=0.5,type='0')

      
   

122.24865591311833

In [19]:
def double_der(f,a,h=1e-6,type='0'):
   if(type=='0'):
      return (f(a+h)+f(a-h)-2*f(a))/(h**2)
   if(type=='-'):
      return (f(a)+f(a-2*h)-2*f(a-h))/(h**2)
   if(type=="+"):
      return (f(a)+f(a+2*h)-2*f(a+h))/(h**2)
def double_der_ae(f,a,h=1e-6,eps_arr=[1e-6,1e-6,1e-6]):
   return (eps_arr[2]+eps_arr[0]-2*eps_arr[1])/(h**2)
def der_interpolation(f,a,h=1,deg=1,type="+"):
   if(type=="+"):
      pass
double_der(lambda x:np.exp(x**2),1.4,0.4)

87.83343403435805

In [20]:
#derivative using guass method::

def f(x1,x2):
    return 2*(x1-x2)/x1     
f(0.0592,0.05918)

0.0006756756756756481

In [43]:
#initial value problem::
#--> make sure model has solution::
A=16
B=8.3
h=0.2
y0,x0=B,1
def f(x,y):
    return A*(x**2)+y**2
def euler(f,x0,y0,x,N=100):
        h=(x-x0)/(N)
        vals=[y0]
        for i in range(N+1):
            vals.append(vals[-1]+h*f(x0+i*h,vals[-1]))
            print(f"y{i+1}:{vals[-1]} x{i+1}:{x0+(i+1)*h}")

def mid_pt(f,x0,y0,x,N=1,pts=None):
    if(pts is None):
        pts=np.array([x0+((x-x0)/N)*i for i in range(N+1)])
    
    vals=np.zeros((10*N))
    vals[0]=y0
    vals[1]=vals[0]+(pts[1]-pts[0])*f(pts[0],vals[0])
    print(f"y1:{vals[1]} x1:{pts[1]}")
    for i in range(1,N,1):
            vals[i+1]=(vals[i-1])+(pts[i+1]-pts[i-1])*f(pts[i],vals[i])
            print(f"y{i+1}:{vals[i+1]} x{i+1}:{pts[i+1]}")
def trapez_de(f,x0,y0,x,N=100):
    h=(x-x0)/(N)
    vals=[y0]
    for i in range(N+1):
        vals.append(vals[-1]+h*0.5*(f(x0+i*h,vals[-1])+f(x0+(i+1)*h,vals[-1])))
        print(f"y{i+1}:{vals[-1]} x{i+1}:{x0+(i+1)*h}")
def i_trapez(f,x0,y0,x,N=100):
    h=(x-x0)/(N)
    vals=[y0]
    for i in range(N+1):
        def fun(y):
             return vals[-1]+h*0.5*(f(x0+i*h,vals[-1])+f(x0+(i+1)*h,y))
        vals.append(fp(lambda x:x,vals[-1]+h*f(x0+i*h,vals[-1]),g=fun,N=1,eps=np.inf))
        #vals.append(vals[-1]+h*0.5*(f(x0+i*h,vals[-1])+f(x0+(i+1)*h,vals[-1])))
        print(f"y{i+1}:{vals[-1]} x{i+1}:{x0+(i+1)*h}")
euler(f,x0,y0,0.4,N=2)
print("==============")
#euler(f,x0,y0,1.01,N=3)
#mid_pt(f,x0,y0,1.01,N=3)
#trapez_de(f,x0,y0,1.01,N=3)
i_trapez(f,x0,y0,0.4,N=2)

#solve(f,x0,y0,x=1,meth=transform_simpsons)

y1:-17.167 x1:0.7
y2:-107.9307667 x2:0.4
y3:-3603.4138868349487 x3:0.10000000000000009
y1:-49.815383350000005 x1:0.7
y2:-95618.69170663922 x2:0.4
y3:-1.1285875035769716e+18 x3:0.10000000000000009


In [44]:
#runge kutta methods::)
#--->use taylor serirs, trucation error vs order of convergence ::

def rk(f,x0,y0,x,N=100):
    h=(x-x0)/(N)
    vals=[y0]
    for i in range(N+1):
        vals.append(vals[-1]+h*0.5*f(x0+i*h,vals[-1])+h*0.5*f(x0+(i+1)*h,vals[-1]+h*f(x0+i*h,vals[-1])))
        print(f"y{i+1}:{vals[-1]} x{i+1}:{x0+(i+1)*h}")
rk(f,x0,y0,1+h,N=2)

y1:27.606026049999997 x1:1.1
y2:626.9974633700599 x2:1.2
y3:79787988.15202355 x3:1.2999999999999998
